# A Playground for a Basic Neural Network

This notebook provides the underlying code for the concepts we saw in the Manim visualization. While the animation gives us the visual intuition, this playground lets you get your hands dirty and see how the numbers actually flow.

We'll build the entire neural network from scratch using only Python and NumPy, focusing on the core mechanics of **feedforward propagation**.

**Goal:** To understand how a given set of inputs, weights, and biases produces a final output.

## Part 1: The Activation Function (The "Squisher")

First, let's define the sigmoid activation function. As we saw in the animation, its job is to take any number and "squish" it into a value between 0 and 1. This is useful for representing things like probabilities or the "activation level" of a neuron.

In [1]:
import numpy as np

def sigmoid(x):
    """This is our activation function. It squishes any value into the (0, 1) range."""
    return 1 / (1 + np.exp(-x))

# Let's test it out!
large_positive_number = 10
large_negative_number = -8
zero = 0

print(f"Sigmoid of {large_positive_number}: {sigmoid(large_positive_number):.4f}") # Should be close to 1
print(f"Sigmoid of {large_negative_number}: {sigmoid(large_negative_number):.4f}") # Should be close to 0
print(f"Sigmoid of {zero}: {sigmoid(zero):.4f}")                 # Should be exactly 0.5

Sigmoid of 10: 1.0000
Sigmoid of -8: 0.0003
Sigmoid of 0: 0.5000


## Part 2: The Neuron

Now, let's model a single neuron. A neuron takes a set of inputs, multiplies each by a corresponding weight, sums them up, adds a bias, and finally applies the activation function.

$$ \text{output} = \sigma \left( \sum (\text{inputs} \cdot \text{weights}) + \text{bias} \right) $$

We'll create a Python class to represent this.

In [2]:
class Neuron:
    """A single neuron with a set of weights and a bias."""
    def __init__(self, weights, bias):
        self.weights = weights
        self.bias = bias
    
    def feedforward(self, inputs):
        """This is where the calculation happens!"""
        # 1. Calculate the weighted sum
        total = np.dot(self.weights, inputs) + self.bias
        
        # 2. Apply the activation function
        return sigmoid(total)

# --- Let's Play! ---

# Recreate the neuron we focused on in the animation.
# It had 9 inputs, so it needs 9 weights.
weights = np.array([-0.5, 2.0, -0.5, -0.5, 2.0, -0.5, -0.5, 2.0, -0.5]) # Weights for detecting a vertical line
bias = -1.0  # <-- Try changing this! A more negative bias makes the neuron harder to activate.

neuron = Neuron(weights, bias)

# This was our input image of a vertical line
vertical_line_input = np.array([0, 1, 0, 0, 1, 0, 0, 1, 0]) # <-- Try changing the input!

# Let's see what the neuron's activation is for this input
output = neuron.feedforward(vertical_line_input)

print(f"The neuron's activation for a vertical line is: {output:.4f}")

# What about for a horizontal line? The activation should be much lower.
horizontal_line_input = np.array([0, 0, 0, 1, 1, 1, 0, 0, 0])
output_horizontal = neuron.feedforward(horizontal_line_input)
print(f"The neuron's activation for a horizontal line is: {output_horizontal:.4f}")

The neuron's activation for a vertical line is: 0.9933
The neuron's activation for a horizontal line is: 0.5000


## Part 3: The Full Network

Now let's assemble our full network. It's just a collection of layers, and each layer is a collection of neurons. We'll create a class for the whole network that connects the layers.

Our network structure is:
- **Input Layer:** 9 neurons (representing the 3x3 grid)
- **Hidden Layer:** 4 neurons
- **Output Layer:** 2 neurons (one for 'P(Vertical)', one for 'P(Horizontal)')

In [3]:
class OurNeuralNetwork:
    """
    A neural network with:
      - 9 inputs
      - a hidden layer with 4 neurons
      - an output layer with 2 neurons
    Each neuron has weights and a bias, which we'll initialize randomly for this example.
    """
    def __init__(self):
        # Set a seed for reproducibility, so we get the same random numbers every time
        np.random.seed(42)
        
        # --- Hidden Layer (4 neurons) ---
        # Each neuron needs 9 weights (one for each input)
        h1_weights = np.random.rand(9)
        h1_bias = np.random.rand()
        self.h1 = Neuron(h1_weights, h1_bias)
        
        h2_weights = np.random.rand(9)
        h2_bias = np.random.rand()
        self.h2 = Neuron(h2_weights, h2_bias)

        h3_weights = np.random.rand(9)
        h3_bias = np.random.rand()
        self.h3 = Neuron(h3_weights, h3_bias)

        h4_weights = np.random.rand(9)
        h4_bias = np.random.rand()
        self.h4 = Neuron(h4_weights, h4_bias)
        
        # --- Output Layer (2 neurons) ---
        # Each neuron needs 4 weights (one for each output from the hidden layer)
        o1_weights = np.random.rand(4)
        o1_bias = np.random.rand()
        self.o1 = Neuron(o1_weights, o1_bias)
        
        o2_weights = np.random.rand(4)
        o2_bias = np.random.rand()
        self.o2 = Neuron(o2_weights, o2_bias)
        
    def feedforward(self, x):
        # 1. Get the outputs from the hidden layer
        out_h1 = self.h1.feedforward(x)
        out_h2 = self.h2.feedforward(x)
        out_h3 = self.h3.feedforward(x)
        out_h4 = self.h4.feedforward(x)
        
        # This is the input for the output layer
        hidden_layer_output = np.array([out_h1, out_h2, out_h3, out_h4])
        
        # 2. Get the final outputs from the output layer
        out_o1 = self.o1.feedforward(hidden_layer_output)
        out_o2 = self.o2.feedforward(hidden_layer_output)
        
        return np.array([out_o1, out_o2])

# Create an instance of our network
network = OurNeuralNetwork()

# --- Let's Test the Whole Network! ---

# Input 1: The vertical line
vertical_line = np.array([0, 1, 0, 0, 1, 0, 0, 1, 0])
prediction_vertical = network.feedforward(vertical_line)
print(f"Prediction for Vertical Line: {prediction_vertical}")
print(f"(Output 1: P(Vertical), Output 2: P(Horizontal))\n")

# Input 2: The horizontal line
horizontal_line = np.array([0, 0, 0, 1, 1, 1, 0, 0, 0])
prediction_horizontal = network.feedforward(horizontal_line)
print(f"Prediction for Horizontal Line: {prediction_horizontal}\n")

# Input 3: A diagonal line
diagonal_line = np.array([1, 0, 0, 0, 1, 0, 0, 0, 1])
prediction_diagonal = network.feedforward(diagonal_line)
print(f"Prediction for Diagonal Line: {prediction_diagonal}\n")

Prediction for Vertical Line: [0.83192724 0.87373214]
(Output 1: P(Vertical), Output 2: P(Horizontal))

Prediction for Horizontal Line: [0.83351873 0.87197101]

Prediction for Diagonal Line: [0.83198452 0.8734924 ]



## A Final Thought: Where's the "Learning"?

You probably noticed that the network's predictions are pretty random. The first output isn't always highest for the vertical line, and the second isn't always highest for the horizontal line. **This is expected!**

Why? Because we initialized its weights and biases with *random* numbers. The network hasn't learned anything yet.

The process of **training** a neural network is the process of showing it many examples (e.g., thousands of vertical and horizontal lines) and systematically adjusting the weights and biases so that its predictions get better and better.

That process, often done with algorithms like **backpropagation** and **gradient descent**, is the story for another time. For now, you have a solid, hands-on understanding of how an already-trained network makes a prediction.